<a href="https://colab.research.google.com/github/Sharkar96/BHDQN/blob/main/BHDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random as r
import numpy as np
import math as m
np.set_printoptions(precision=2)

In [ ]:

class Obstacle:
  def __init__(self, o, A):
    self.A=A
    self.a=o
    self.createObstacle(o, A)

  #create an obs, first vert is orgini then on right,
  #the fourth is down from origin
  def createObstacle(self, o, A):
    th=m.radians(360*r.random())
    R=np.array([[m.cos(th), -m.sin(th)],[m.sin(th), m.cos(th)]])
    rngLength=r.randint(1, A)

    self.b=np.array([o[0]+rngLength,o[1]])
    self.c=np.array([o[0]+rngLength,o[1]-A/rngLength])
    self.d=np.array([o[0],o[1]-A/rngLength])

    self.b= np.matmul(R, self.b)
    self.c= np.matmul(R, self.c)
    self.d= np.matmul(R, self.d)

  def __str__(self):
    return 'a=' + str(self.a) + ' b=' + str(self.b) + ' c=' + str(self.c) + ' d='+ str(self.d)

class Maze:
  def __init__(self, A):
    self.coordinates=










a=[0 0] b=[0.11 0.99] c=[9.06 0.04] d=[ 8.95 -0.96]
